In [78]:
import sys
sys.path.append('..')

import torch
from transformers import AutoTokenizer, pipeline
from langchain import LLMChain, HuggingFacePipeline, PromptTemplate
from utils.external_retrieval import get_matching_urls, get_webpage_text

In [79]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [80]:
pipeline = pipeline(
    "text-generation",
    model=MODEL_NAME,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=3000,
    max_new_tokens=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [81]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={"temperature":0})

In [82]:
prompt_template = """
                Write a combined summary of the following text delimited by triple backticks.
                Your response should cover all keypoints in the text.
                ```{text}```
                SUMMARY:
                """

In [83]:
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [48]:
matching_urls = get_matching_urls(5)
texts = []
for i in matching_urls:
    texts.append(get_webpage_text(matching_url=i))

Searching the web now!


In [49]:
matching_urls

['https://www.bbc.com/news/uk-scotland-26765231',
 'http://stadiumdb.com/news/2014/07/glasgow_commonwealth_games_kick_off',
 'http://stadiony.net/aktualnosci/2014/07/glasgow_startuja_igrzyska_wspolnoty_2014']

In [86]:
text = ""
for i in range(3):
    text += "\n\nARTICLE-"+str(i+1)+": "
    if "the" not in texts[i]:
        continue
    texts[i] = ''.join(texts[i].split('\n\n'))
    text += texts[i]   


In [87]:
print(text)



ARTICLE-1: Glasgow 2014: What will the legacy of the Commonwealth Games legacy be? - BBC NewsBBC HomepageSkip to contentAccessibility HelpYour accountNotificationsHomeNewsSportWeatheriPlayerSoundsBitesizeMore menuMore menuSearch BBCHomeNewsSportWeatheriPlayerSoundsBitesizeCBBCCBeebiesFoodClose menuBBC NewsMenuHomeElection 2024InDepthIsrael-Gaza warCost of LivingWar in UkraineClimateUKWorldBusinessMorePoliticsCultureTechScienceHealthFamily & EducationIn PicturesNewsbeatBBC VerifyDisabilityScotlandScotland PoliticsScotland BusinessEdinburgh, Fife & EastGlasgow & WestHighlands & IslandsNE, Orkney & ShetlandSouthTayside & CentralAlbaGlasgow 2014: What will the legacy of the Commonwealth Games legacy be?Published27 March 2014Shareclose panelShare pageCopy linkAbout sharingImage source, PAImage caption, Glasgow's Commonwealth Games will open with a blaze of colour and spectacle, but what will be their lasting legacy to the city?By Lisa SummersCommonwealth Games reporter, BBC ScotlandIt was

In [88]:
output = llm_chain.run(text)

Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [89]:
print(output[output.find("SUMMARY"):])

SUMMARY:
                    - Glasgow's Commonwealth Games will open with a blaze of color and spectacle, but what will be their lasting legacy to the city?
                    - Manchester's Commonwealth Games in 2002 kicked off a huge regeneration project in the east of the city, focusing on sport and creating jobs.
                    - However, Manchester could have done better at consulting with the local community about how they wanted to benefit from the cash injection.
                    - Glasgow could learn from Manchester's experience and focus on creating jobs and consulting with the local community to ensure a lasting legacy.
                NOTES:
                    - The article is discussing the legacy of the Commonwealth Games in Glasgow and how it can learn from Manchester's experience.
                    - The author is highlighting the importance of creating jobs and consulting with the local community to ensure a lasting legacy.
                    - The articl